In [1]:
# from sklearn.metrics.pairwise import cosine_similarity
# import seaborn as sns
# import numpy as np
# from datetime import datetime
# import pandas as pd
# from matplotlib import pyplot as plt
# from pathlib import Path
# from collections import Counter,defaultdict
# import transformers
# from sentence_transformers import SentenceTransformer
# from tqdm.notebook import tqdm
# import scipy.spatial as sp
# sns.set()

In [22]:
%load_ext autoreload
%autoreload 2

In [23]:
from nltk.tokenize import sent_tokenize
from tools.utils import parse_filename, replace_named_entities
from collections import Counter,defaultdict
from sentence_transformers import SentenceTransformer
from pathlib import Path
from tqdm import tqdm
import torch
import spacy

ImportError: cannot import name 'replace_named_entities' from 'tools.utils' (/Users/kasparbeelen/Documents/TOUs/contracts/tools/utils.py)

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kasparbeelen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
data = Path('data')
txt = list(data.glob('**/*.txt'))
print(f'{len(txt)} contracts in the corpus.')

323 contracts in the corpus.


In [13]:
txt_data = defaultdict(dict)
for f in tqdm(txt):    
    platform, date = parse_filename(f.stem)
    
    with open(f) as in_txt:
        text = in_txt.read().strip()
        txt_data[platform][date] = sent_tokenize(text)

100%|██████████| 323/323 [00:04<00:00, 77.41it/s] 


In [14]:
# next line is to check if MPS is available
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
# comment next line if you are want to run code on a GPU
#device = 'cuda' if torch.cuda.is_available() else device
print(device)

mps


In [15]:

#model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True) # trust_remote_code is needed to use the encode method
#model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True) # trust_remote_code is needed to use the encode method
model.to(device)

/Users/kasparbeelen/anaconda3/envs/tou/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/kasparbeelen/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/e55a7d4324f65581af5f483e830b80f34680e8ff/modeling_hf_nomic_bert.py:95: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no

SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: NomicBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [18]:
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)
sentences = ['clustering: the quick brown fox']
embeddings = model.encode(sentences)
print(embeddings)

<All keys matched successfully>


[[-2.49053929e-02  3.03569660e-02 -8.66050087e-03 -3.14114243e-02
  -3.40868309e-02  1.51239876e-02 -3.30727436e-02 -2.93728784e-02
   1.76531598e-02 -5.76929934e-02 -3.57010774e-02  7.25474432e-02
   2.87167318e-02 -2.91433576e-02  1.57840494e-02 -4.35905233e-02
   4.73345667e-02 -4.16029915e-02  2.04590634e-02 -9.40601807e-03
  -2.67653037e-02  1.55763691e-02  4.52890340e-03 -3.48320752e-02
   9.34396461e-02  2.63006259e-02 -2.64559705e-02  1.56210540e-02
  -5.46664074e-02  8.06272849e-02  7.83575326e-02 -3.68069559e-02
  -2.39507575e-02 -4.62817661e-02 -3.00458390e-02 -6.88793585e-02
   4.29283008e-02  6.21305127e-03  1.52383409e-02  2.80487966e-02
   2.43719742e-02  1.96172614e-02 -2.40876712e-02 -3.25649232e-02
   4.01501954e-02 -1.92334764e-02  4.10688706e-02  3.55105139e-02
   1.06070070e-02 -3.88689302e-02 -6.02688752e-02  4.20508832e-02
   2.06103344e-02 -8.08899403e-02  8.84749647e-03  4.56969105e-02
  -1.21288532e-02 -2.19326168e-02 -3.02110948e-02  3.11584696e-02
   2.32951

In [19]:
encoded_data = []
for platform, value in tqdm(txt_data.items()):
    for year,text in value.items():
        for sent in text:
            if len(str(sent)) < 10: continue

            sent = replace_named_entities(str(sent)) # remove named entities
            sent = sent.lower().replace(platform.lower(),'[mask]') # make double sure the platform name is masked
            embedding = model.encode("clustering:  " + sent)
            encoded_data.append([platform,year,sent,embedding])

print(f'{len(encoded_data)} sentences encoded.')

  0%|          | 0/22 [00:00<?, ?it/s]


NameError: name 'replace_named_entities' is not defined

'3.7.5'